In [1]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import importlib
import sqlite3
import numpy as np

# keeping company information in additional file
import data_file

In [442]:
#combining products groups 
control = data_file.control
transmitters = data_file.transmitters
flowmeters = data_file.flowmeters
gc = data_file.gc
analytical = data_file.analytical
asi = data_file.asi
other_prj_product = data_file.other_prj_product

all_groups = control + transmitters + flowmeters + gc + analytical + asi + other_prj_product

In [443]:
# Define a function to combine values in bu
def combine_values(x):
    if x in control:
        return f'Control {control}'
    elif x in transmitters:
        return f'Transmitters {transmitters}'
    elif x in flowmeters:
        return f'Flowmeters {flowmeters}'
    elif x in gc:
        return f'GC {gc}'
    elif x in analytical:
        return f'Analytical Products {analytical}'
    elif x in asi:
        return f'ASI {asi}'
    elif x in other_prj_product:
        return f'Other prj products {other_prj_product}'
    else:
        return 'Other'

# Apply the function to create a new column with combined values
#df['Business Unit'] = df['bu'].apply(combine_values)

In [93]:
save_copy = df_orders.copy()

In [79]:
end_customers = save_copy[['company_code_n','ec_eu_customer', 'ec_eu_customer_n', 'ec_eu_country_n']]

In [81]:
end_customers['ec_eu_customer'] = end_customers['ec_eu_customer'].astype(str)
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
all_cust = df_customers['sold_to_customer'].to_list()

end_customers = end_customers[~end_customers['ec_eu_customer'].isin(all_cust)]

end_customers = end_customers.drop_duplicates()

# Save to Excel
end_customers.to_excel('data_files/outcome/output.xlsx', index=False)

In [412]:
df_orders = save_copy.copy()

In [92]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
conn.close()

sum_before = df_orders.order_intake_amount_eur.sum()
df_orders = df_orders[df_orders['order_intake_amount_eur'] != 0]
sum_after = df_orders.order_intake_amount_eur.sum()

check = sum_after - sum_before
print(round(check, 2))

-0.0


In [413]:
# Cleaning of industry information
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == '(NOT USE) POWER : HYDRO', 'eu_industry_segment_n_latest'] = 'RENEWABLE ENERGY : HYDRO'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'RENEWABLE POWER: SOLAR, WIND, HYDRO, WTE, BIOMASS AND OTHERS', 'eu_industry_segment_n_latest'] = 'RENEWABLE ENERGY: SOLAR, WIND, HYDRO, WTE, BIOMASS AND OTHERS'


# Create a new column based on the 'industry' column
df_orders['ec_eu_industry_n'] = df_orders['ec_eu_industry_n'].str.upper()
df_orders['eu_industry_segment_n_latest'] = df_orders['eu_industry_segment_n_latest'].str.upper()
df_orders['eu_industry_segment_n_latest'] = df_orders['eu_industry_segment_n_latest'].str.replace('：',':')

# Create the 'industry' column using numpy.where
df_orders['industry'] = np.where( df_orders['eu_industry_segment_n_latest'].str.contains(':'),
                                  df_orders['eu_industry_segment_n_latest'].str.split(':').str[0].str.strip(),
                                  df_orders['ec_eu_industry_n'])

df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'PULP & PAPER', 'industry'] = 'PULP & PAPER'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'MINING (EXCEPT OIL AND GAS)', 'industry'] = 'METALS, MINING AND MINERALS PROCESSING'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'NON-FERROUS METAL MANUFACTURING', 'industry'] = 'METALS, MINING AND MINERALS PROCESSING'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'NONFERROUS METAL (EXCEPT ALUMINUM) PRODUCTION AND PROCESSING', 'industry'] = 'METALS, MINING AND MINERALS PROCESSING'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'FABRICATED METAL PRODUCT MANUFACTURING', 'industry'] = 'METALS, MINING AND MINERALS PROCESSING'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'ALUMINA AND ALUMINUM PRODUCTION AND PROCESSING', 'industry'] = 'METALS, MINING AND MINERALS PROCESSING'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'PHARMACEUTICAL', 'industry'] = 'PHARMACEUTICAL'
df_orders.loc[df_orders['eu_industry_segment_n_latest'] == 'MEDICAL EQUIPMENT AND SUPPLIES MANUFACTURING', 'industry'] = 'PHARMACEUTICAL'

In [414]:
df_orders = df_orders[['company_code_n', 'FY', 'bu2', 'sales_order_so', 'sold_to_customer', 'ec_eu_customer_n', 'industry','order_intake_amount_eur']]

In [415]:
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers" 
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [ ]:
df_customers  = df_customers[['sold_to_customer', 'customer_name', 'type', 'tier_new', 'countries']]
df_customers.columns  = ['sold_to_customer', 'customer_name', 'type', 'business type', 'Country']

df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
df_orders['sold_to_customer'] = df_orders['sold_to_customer'].astype(str)
merged_with_types = df_orders.merge(df_customers, on='sold_to_customer', how='left')

print(len(merged_with_types))

merged_with_types['business type'] = merged_with_types['business type'].fillna('Direct')

merged_with_types['FY'] = pd.to_datetime(merged_with_types['FY'])

merged_with_types['year'] = merged_with_types['FY'].dt.year

In [446]:
# Convert columns to numeric
years = list(range(2018, 2023))
period_of_analysis = len(years)

def determine_trend(row):
    values = row[years].values.astype(float)  # Ensure values are floats
    slope, _ = np.polyfit(years, values, 1)  # linear fit (degree=1)

    if slope > 0:
        trend = "positive"
    else:
        trend = "negative"

    # Here, 5 is just a sample threshold for strong/weak distinction, adjust as needed
    #magnitude = "strong" if abs(slope) > 5 else "weak" {magnitude}

    return f"{trend}"

def determine_slope(row):
    values = row[years].values.astype(float)  # Ensure values are floats
    slope, _ = np.polyfit(years, values, 1)  # linear fit (degree=1)
    
    return slope

In [447]:
def trend_data(df_init, opco):
        
    if opco != 'all':
        df = df_init[df_init['company_code_n'] == opco].copy()
    elif opco == 'all':
        df = df_init.copy()
    else:
        df = df_init[df_init['Country'] == 'Norway'].copy()
    
    
    # number of customers
    # Get unique sold_to_customer count
    grouped = df.groupby(['business type', 'year']).agg(unique_customers=('sold_to_customer', 'nunique')).reset_index()
    # Pivot the table
    pivot_df = grouped.pivot_table(index=['business type'],
                               columns='year',
                               values='unique_customers',
                               aggfunc='sum').reset_index()
    
    pivot_df.fillna(0, inplace=True)
     
    # Add 'Total' row
    total_row = pivot_df.sum(numeric_only=True)  # sum only numeric columns
    total_row['business type'] = 'All'  # set 'type' column to 'All'
    pivot_df = pd.concat([pivot_df, pd.DataFrame([total_row])], ignore_index=True) #
    
    
    pivot_df['Trend'] = pivot_df.apply(determine_trend, axis=1)
    #pivot_df['Slope'] = pivot_df.apply(determine_slope, axis=1)
    
    number_of_customers = pivot_df.copy()
    
    # industry trends
    # Group by the columns and aggregate with sum
    grouped = df.groupby(['company_code_n', 'year', 'industry'])['order_intake_amount_eur'].sum().reset_index()
    # Optionally, pivot the table for a clearer view
    pivot_df = grouped.pivot_table(index=['company_code_n', 'industry'], columns='year', values='order_intake_amount_eur', aggfunc='sum').reset_index()
    
    pivot_df.fillna(0, inplace=True)
    
    # Add the 'Total' column
    pivot_df['Total'] = pivot_df.sum(axis=1, numeric_only=True)

    pivot_df['Trend'] = pivot_df.apply(determine_trend, axis=1)
    #pivot_df['Slope'] = pivot_df.apply(determine_slope, axis=1)
    
    
    # Sort customers by Total in descending order
    result = pivot_df.sort_values(by='Total', ascending=False)

    # Calculate cumulative sum and percentage of total
    result['Cumulative_Sum'] = result['Total'].cumsum()
    result['Cumulative_Percent'] = 100 * result['Cumulative_Sum'] / result['Total'].sum()

    # Assign categories based on cumulative percentage
    result['Category'] = pd.cut(
        result['Cumulative_Percent'], 
        bins=[0, 80, 95, 100], 
        labels=['A', 'B', 'C']
    )

    # Drop temporary columns
    result.drop(['Cumulative_Sum', 'Cumulative_Percent'], axis=1, inplace=True)

    industry_trends = result.copy()
    
    
    # Filter the DataFrame for category 'A'
    category_a_df = industry_trends[industry_trends['Category'] == 'A']

    # Group industries by trend
    grouped = category_a_df.groupby('Trend')['industry'].apply(list)

    # Construct the text
    text_parts = []
    for trend, industries in grouped.items():
        industries_str = ', '.join(industries)
        text_parts.append(f"{industries_str} demonstrate a {trend} trend.")

    text_industry = " ".join(text_parts)

    #product trends
    # Group by the columns and aggregate with sum
    grouped = df.groupby(['company_code_n', 'year', 'bu2'])['order_intake_amount_eur'].sum().reset_index()
    pivot_df = grouped.pivot_table(index=['company_code_n', 'bu2'], columns='year', values='order_intake_amount_eur', aggfunc='sum').reset_index()
    pivot_df.fillna(0, inplace=True)
    
    # Add the 'Total' column
    pivot_df['Total'] = pivot_df.sum(axis=1, numeric_only=True)
    
    pivot_df['Trend'] = pivot_df.apply(determine_trend, axis=1)
    #pivot_df['Slope'] = pivot_df.apply(determine_slope, axis=1)
    

    # Sort customers by Total in descending order
    result = pivot_df.sort_values(by='Total', ascending=False)

    # Calculate cumulative sum and percentage of total
    result['Cumulative_Sum'] = result['Total'].cumsum()
    result['Cumulative_Percent'] = 100 * result['Cumulative_Sum'] / result['Total'].sum()

    # Assign categories based on cumulative percentage
    result['Category'] = pd.cut(
        result['Cumulative_Percent'], 
        bins=[0, 80, 95, 100], 
        labels=['A', 'B', 'C']
    )

    # Drop temporary columns
    result.drop(['Cumulative_Sum', 'Cumulative_Percent'], axis=1, inplace=True)
    
    product_trends = result.copy()
    
    # Filter the DataFrame for category 'A'
    category_a_df = product_trends[product_trends['Category'] == 'A']

    # Group industries by trend
    grouped = category_a_df.groupby('Trend')['bu2'].apply(list)

    # Construct the text
    text_parts = []
    for trend, bu in grouped.items():
        bu_str = ', '.join(bu)
        text_parts.append(f"{bu_str} demonstrate a {trend} trend.")

    text_bu = " ".join(text_parts)
    
    text_info = [f'From TOP industries for the last {period_of_analysis} years\n' + text_industry , f'From TOP product bu for the last {period_of_analysis} years\n' + text_bu]
   
    return number_of_customers, industry_trends, product_trends, text_info

In [448]:
# Define a custom function to join non-'OTHERS' industries if present
def custom_join(x):
    non_others = [industry for industry in x if industry != 'OTHERS']
    return '; '.join(sorted(set(non_others))) if non_others else 'OTHERS'

# Apply the custom function within transform
merged_with_types['industry_comb'] = merged_with_types.groupby('customer_name')['industry'].transform(custom_join)

In [449]:
target_bu = data_file.target_bu

In [450]:
# Function to categorize customers
def categorize_customer(row):
    recent_years = [2021, 2022, 2023]
    # Extract only the columns for years
    year_data = row[[2017, 2018, 2019, 2020, 2021, 2022, 2023]]
    
    # Get years with purchases
    purchase_years = year_data[year_data > 0].index.tolist()
    
    if len(purchase_years) >= 4 and (2022 in purchase_years or 2023 in purchase_years):
        return 'Active'
    elif not set(recent_years).intersection(purchase_years):
        return 'Sleeping'
    else:
        return 'Sporadic'

In [451]:
def soyears(temp, q=''):
    if q == 'bu':
        suf = '_bu'
    else:
        suf = ''
    
    df = temp.copy()
    
    # FY 23 is not full yet
    df = df[df['FY'] < 2023]

    # 1. Calculate average number of unique 'sales_order_so' per year ('FY') for each 'customer_name'
    grouped1 = (temp.groupby(['customer_name', 'FY'])
            .agg(unique_so_per_FY=pd.NamedAgg(column='sales_order_so', aggfunc='nunique'))
            .reset_index()
            .groupby('customer_name')
            .agg(so_number_per_FY=pd.NamedAgg(column='unique_so_per_FY', aggfunc='mean'))
            .reset_index())
    
    # 2. Calculate the average amount of 'order_intake_amount_eur' for each 'sales_order_so'
    grouped2 = (temp.groupby(['customer_name', 'sales_order_so'])
            .agg(mean_amount=pd.NamedAgg(column='order_intake_amount_eur', aggfunc='sum'))
            .reset_index()
            .groupby('customer_name')
            .agg(so_average_in_eur=pd.NamedAgg(column='mean_amount', aggfunc='mean'))
            .reset_index())
    
    # 3. Merge the results
    result_df = pd.merge(grouped1, grouped2, on='customer_name', how='inner') 
    result_df.columns = ['customer_name', f'so_number_per_FY{suf}', f'so_average_in_eur{suf}']
       
    return result_df


def anupamtenyears(opco, df_init):
    if opco != 'all':
        df = df_init[df_init['company_code_n'] == opco].copy()
    elif opco == 'all':
        df = df_init.copy()
    else:
        df = df_init[df_init['Country'] == 'Norway'].copy()
    
    df['FY'] = pd.to_datetime(df['FY'])
    df['FY'] = df['FY'].dt.year
    
    df['bu2'] = df['bu2'].astype(str)
    df_target_bu = df[df['bu2'].isin(target_bu)]
    
    so_df = soyears(df)
    so_df_bu = soyears(df_target_bu, q='bu')
    
    # Update the values in 'bu2' column if they are in the list
    df['bu2'] = df['bu2'].apply(lambda x: 'PCI_' + x if x in target_bu else x)
    
    
    #print(df.order_intake_amount_eur.sum())
    #print(df_target_bu.order_intake_amount_eur.sum())
    
    
    # Create the pivot table
    pivot_df = df.pivot_table(index=['customer_name','Country','type', 'business type', 'industry_comb'], columns='FY', values='order_intake_amount_eur', aggfunc='sum', fill_value=0) 
    pivot_df_target = df_target_bu.pivot_table(index=['customer_name', 'Country','type', 'business type', 'industry_comb'], columns='FY', values='order_intake_amount_eur', aggfunc='sum', fill_value=0) 
    pivot_df_bu = df.pivot_table(index=['customer_name', 'Country', 'type', 'business type', 'industry_comb','FY'], columns='bu2', values='order_intake_amount_eur', aggfunc='sum', fill_value=0)

    
    # Add the 'Total' column
    pivot_df['Total'] = pivot_df.sum(axis=1)
    pivot_df_target['Total'] = pivot_df_target.sum(axis=1)
    
    pivot_df = pivot_df[['Total']]
    pivot_df.columns = ['Total_Total']
    
        
    result = pivot_df.merge(pivot_df_target, left_index=True, right_index=True, how='left')
    result['Yoko Product Share'] = result['Total'] / result['Total_Total']
    result['Yoko Product Share'].fillna(0, inplace=True)
    result = result[[2017, 2018, 2019, 2020, 2021, 2022, 2023, 'Total', 'Total_Total', 'Yoko Product Share']]
    result = result.sort_values(by=['Total_Total', 'Yoko Product Share'], ascending=False)
    result.fillna(0, inplace=True)
    result.reset_index(inplace=True)
    
    result = result.merge(so_df, how='left', on='customer_name')
    result = result.merge(so_df_bu, how='left', on='customer_name')
    
    pivot_df_bu['Total'] = pivot_df_bu.sum(axis=1)
    
    
    # Sort customers by Total in descending order
    result = result.sort_values(by='Total_Total', ascending=False)

    # Calculate cumulative sum and percentage of total
    result['Cumulative_Sum'] = result['Total_Total'].cumsum()
    result['Cumulative_Percent'] = 100 * result['Cumulative_Sum'] / result['Total_Total'].sum()

    # Assign categories based on cumulative percentage
    result['Category'] = pd.cut(
        result['Cumulative_Percent'], 
        bins=[0, 80, 95, 100], 
        labels=['A', 'B', 'C']
    )

    # Drop temporary columns
    result.drop(['Cumulative_Sum', 'Cumulative_Percent'], axis=1, inplace=True)
    
    # Sort customers by Total in descending order
    result = result.sort_values(by='Total', ascending=False)

    # Calculate cumulative sum and percentage of total
    result['Cumulative_Sum'] = result['Total'].cumsum()
    result['Cumulative_Percent'] = 100 * result['Cumulative_Sum'] / result['Total'].sum()

    # Assign categories based on cumulative percentage
    result['Category Product BU'] = pd.cut(
        result['Cumulative_Percent'], 
        bins=[0, 80, 95, 100], 
        labels=['A', 'B', 'C']
    )

    # Drop temporary columns
    result.drop(['Cumulative_Sum', 'Cumulative_Percent'], axis=1, inplace=True)
    
   
    result['Activeness'] = result.apply(categorize_customer, axis=1)
    
    
    new_customers_per_year = {}
    
    for year in df['year'].unique():
        # Customers up to last year
        previous_customers = df[df['year'] < year]['customer_name'].unique()

        # Customers this year
        customers_this_year = df[df['year'] == year]['customer_name'].unique()

        # New customers this year (set difference)
        new_customers = set(customers_this_year) - set(previous_customers)

        # Store the count of new customers
        new_customers_per_year[year] = len(new_customers)
    
    # Convert the result to a dataframe
    new_customers_df = pd.DataFrame(list(new_customers_per_year.items()), columns=['Year', 'New Customers'])

    # Transform the dataframe
    pivot_df = new_customers_df.pivot(index=None, columns='Year', values='New Customers').reset_index(drop=True)
    pivot_df = pivot_df[sorted(pivot_df.columns)]  # Sort columns in ascending order
    
    # Sum across rows
    summed_row = pivot_df.sum(axis=0)

    # Convert Series to DataFrame and reshape
    final_df = summed_row.to_frame().T

    # Set the index
    final_df.index = ['Number of new customers']
   
    new_customers_df = final_df.copy()
    
    
    new_customers_per_year_bu = {}

    for year in df_target_bu['year'].unique():
        # Customers up to last year
        previous_customers = df_target_bu[df_target_bu['year'] < year]['customer_name'].unique()
        # Customers this year
        customers_this_year = df_target_bu[df_target_bu['year'] == year]['customer_name'].unique()
        # New customers this year (set difference)
        new_customers = set(customers_this_year) - set(previous_customers)
        # Store the count of new customers
        new_customers_per_year_bu[year] = len(new_customers)
    
    # Convert the result to a dataframe
    new_customers_bu_df = pd.DataFrame(list(new_customers_per_year_bu.items()), columns=['Year', 'New Customers'])

    # Transform the dataframe
    pivot_df = new_customers_bu_df.pivot(index=None, columns='Year', values='New Customers').reset_index(drop=True)
    pivot_df = pivot_df[sorted(pivot_df.columns)]  # Sort columns in ascending order
    
    # Sum across rows
    summed_row = pivot_df.sum(axis=0)

    # Convert Series to DataFrame and reshape
    final_df = summed_row.to_frame().T

    # Set the index
    final_df.index = ['Number of new customers for Product']
   
    new_customers_df = pd.concat([final_df, new_customers_df])
    
    
    # Filter the dataframe based on conditions
    active_top_customers = result[ (result['Category Product BU'] == 'A') & 
                     (result['Activeness'] == 'Active')]
    
   
    # Filter the dataframe based on conditions
    sleeping_top_customers = result[(result['Category Product BU'] == 'A') & 
                     (result['Activeness'] == 'Sleeping')]
    
    
    active_distributors = result[(result['type'] == 'Distributor') & (result['Activeness'] == 'Active')]

    
    number_info = [len(active_top_customers), len(sleeping_top_customers), len(active_distributors)]
    
   
    return result, pivot_df_bu, new_customers_df, number_info

In [ ]:
# all opcos preparation
opco_list = ['all']
for_trend_df = merged_with_types[merged_with_types['bu2'].isin(target_bu)]

for_trend_df['bu2'] = for_trend_df['bu2'].apply(combine_values)

for opco in opco_list:

    df_customer_count = trend_data(for_trend_df, opco)[0]
    df_industry = trend_data(for_trend_df, opco)[1]
    df_products = trend_data(for_trend_df, opco)[2]
    
    print(trend_data(for_trend_df, opco)[3][0], '\n' + trend_data(for_trend_df, opco)[3][1])
    
    df_for_excel = anupamtenyears(opco, merged_with_types)[0]
    df_target_for_excel = anupamtenyears(opco, merged_with_types)[1]
    new_cust = anupamtenyears(opco, merged_with_types)[2]
    info_text = anupamtenyears(opco, merged_with_types)[3]
    
    print(f'Number of active TOP customers of Yokogawa-Product for {opco} is {info_text[0]}\nNumber of sleeping TOP customers of Yokogawa-Product is {info_text[1]}\nNumber of active distributors {info_text[2]} ')
    
    writer = pd.ExcelWriter(f'data_files/outcome/{opco}_overview_18.xlsx')
    df_customer_count.to_excel(writer, sheet_name='customer count', index=True)
    df_industry.to_excel(writer, sheet_name='industry trends', index=True)
    df_products.to_excel(writer, sheet_name='products trends', index=True)
    
    df_for_excel.to_excel(writer, sheet_name='years', index=True)
    df_target_for_excel.to_excel(writer, sheet_name='bu_years', index=True)
    new_cust.to_excel(writer, sheet_name='new_customers', index=True)
    
    writer.close()

In [ ]:
import matplotlib.pyplot as plt
for_plot_df = merged_with_types.copy()

# Get unique countries
opco_list = ['all']

for opco in opco_list:
    # Filter DataFrame for a specific country
    temp_df = for_plot_df[for_plot_df['company_code_n'] == opco]
    
    temp_df['order_intake_amount_eur'] = round((temp_df['order_intake_amount_eur'] / 1000) , 0)

    # Group by industry and get the sum of order intake amount
    grouped = temp_df.groupby('industry')['order_intake_amount_eur'].sum().sort_values(ascending=False)

    # Calculate the total sum for labeling purposes
    total_sum = grouped.sum()

    # Plotting with larger figure size
    plt.figure(figsize=(12, 8))
    ax = grouped.plot(kind='bar', color=plt.cm.Paired.colors)
    ax.set_title(f"Order Intake Amount per Industry for {opco}", fontsize=16)
    ax.set_ylabel("Amount (in kEuro)", fontsize=14)
    #ax.set_xlabel("Industry", fontsize=8)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Set the x-tick labels (industries) to be horizontal and adjust font size if needed
    plt.xticks(fontsize=8, rotation=45)  # Set rotation to 0 for horizontal labels
    
    # Add labels to the bars with the absolute value and percentage
    for rect in ax.patches:
        height = rect.get_height()
        ax.annotate(f'{height}k€ ({height/total_sum:.0%})',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),  # 5 points vertical offset for better spacing
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

    # Remove top, left, and right frame, but keep the x-axis (bottom spine)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    # Save the plot as a .jpg file
    plt.savefig(f'data_files/outcome/{opco}_industry_sales.jpg', format='jpg', dpi=300)

    plt.close()  # Close the plot to free up memory

In [65]:
# discont analysis
discounts_df = pd.read_excel('data_files/discounts.xlsx')

discounts_df['sstp_approval_no'] = discounts_df['sstp_approval_no'].notna().astype(int)

In [ ]:
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
discounts_df['sold_to_customer'] = discounts_df['sold_to_customer'].astype(str)
discounts_df['sales_order_so'] = discounts_df['sales_order_so'].astype(str)
discounts_with_customer_name = discounts_df.merge(df_customers, on='sold_to_customer', how='left')

print(len(discounts_with_customer_name))

discounts_with_customer_name['business type'] = discounts_with_customer_name['business type'].fillna('Direct')
discounts_with_customer_name['eu_industry_segment_n_latest'] = discounts_with_customer_name['eu_industry_segment_n_latest'].fillna('Others')

discounts_with_customer_name = discounts_with_customer_name[~discounts_with_customer_name['type'].isna()]

wdf = discounts_with_customer_name.copy()

wdf = wdf[['customer_name', 'type', 'business type', 'Country', 'fiscal_year', 'year_month', 'sales_order_so',
       'sstp_approval_no', 'eu_industry_segment_n_latest', 'bu', 'material', 'ms_code',
       'sales_quantity', 'sales_rsp_eur', 'sales_eur', 'cogs(ms)_eur',
       'gp_eur']]

wdf['FY'] = wdf['fiscal_year']

In [112]:
pivot_wdf_qty = wdf.pivot_table(index=['customer_name','Country','type', 'business type'], columns='FY', values='sales_quantity', aggfunc='sum', fill_value=0) 
pivot_wdf_sales_rsp = wdf.pivot_table(index=['customer_name','Country','type', 'business type'], columns='FY', values='sales_rsp_eur', aggfunc='sum', fill_value=0)
pivot_wdf_sales = wdf.pivot_table(index=['customer_name','Country','type', 'business type'], columns='FY', values='sales_eur', aggfunc='sum', fill_value=0)
pivot_wdf_cogs = wdf.pivot_table(index=['customer_name','Country','type', 'business type'], columns='FY', values='cogs(ms)_eur', aggfunc='sum', fill_value=0)
pivot_wdf_gp = wdf.pivot_table(index=['customer_name','Country','type', 'business type'], columns='FY', values='gp_eur', aggfunc='sum', fill_value=0)

bu_pivot_wdf_qty = wdf.pivot_table(index='bu', columns='FY', values='sales_quantity', aggfunc='sum', fill_value=0) 
bu_pivot_wdf_sales_rsp = wdf.pivot_table(index='bu', columns='FY', values='sales_rsp_eur', aggfunc='sum', fill_value=0)
bu_pivot_wdf_sales = wdf.pivot_table(index='bu', columns='FY', values='sales_eur', aggfunc='sum', fill_value=0)
bu_pivot_wdf_cogs = wdf.pivot_table(index='bu', columns='FY', values='cogs(ms)_eur', aggfunc='sum', fill_value=0)
bu_pivot_wdf_gp = wdf.pivot_table(index='bu', columns='FY', values='gp_eur', aggfunc='sum', fill_value=0)

material_pivot_wdf_qty = wdf.pivot_table(index='material', columns='FY', values='sales_quantity', aggfunc='sum', fill_value=0) 
material_pivot_wdf_sales_rsp = wdf.pivot_table(index='material', columns='FY', values='sales_rsp_eur', aggfunc='sum', fill_value=0)
material_pivot_wdf_sales = wdf.pivot_table(index='material', columns='FY', values='sales_eur', aggfunc='sum', fill_value=0)
material_pivot_wdf_cogs = wdf.pivot_table(index='material', columns='FY', values='cogs(ms)_eur', aggfunc='sum', fill_value=0)
material_pivot_wdf_gp = wdf.pivot_table(index='material', columns='FY', values='gp_eur', aggfunc='sum', fill_value=0)

In [ ]:
# 1. Calculate % Discount:
pivot_wdf_discount = (pivot_wdf_sales_rsp - pivot_wdf_sales) / pivot_wdf_sales_rsp * 100

# 2. Calculate % Profit:
# Here, you might want to handle cases where 'sales_eur' is zero to avoid division by zero errors
with np.errstate(divide='ignore', invalid='ignore'):  # This will handle the divide by zero warning
    pivot_wdf_profit = pivot_wdf_gp / pivot_wdf_sales * 100
    pivot_wdf_profit[pivot_wdf_sales == 0] = np.nan  # Replace any infinity values with NaN or you can use 0 or any other number

In [128]:
# Group by sales_order_so and determine its approval status
approval_status = wdf.groupby('sales_order_so')['sstp_approval_no'].agg(['sum', 'count'])

approval_status['approval_type'] = np.where(
    approval_status['sum'] > 0, 'With Approval', 'Without Approval'
)

# Merge the original dataframe with the approval status
wdf = wdf.merge(approval_status[['approval_type']], left_on='sales_order_so', right_index=True, how='left')

# Now, get counts for each customer_name
total_sales_orders = wdf.groupby('customer_name')['sales_order_so'].nunique()

sales_orders_with_approval = wdf[wdf['approval_type'] == 'With Approval'].groupby('customer_name')['sales_order_so'].nunique()

sales_orders_without_approval = wdf[wdf['approval_type'] == 'Without Approval'].groupby('customer_name')['sales_order_so'].nunique()

# Combine the results
result = pd.concat([total_sales_orders, sales_orders_with_approval, sales_orders_without_approval], axis=1)
result.columns = ['Total Sales Orders', 'With Approval', 'Without Approval']

In [130]:
writer = pd.ExcelWriter(f'data_files/outcome/check_.xlsx')
result.to_excel(writer, sheet_name=f'years', index=True)
writer.close()

In [92]:
df = wdf.copy()

# 2. Time Series Analysis:
monthly_sales = df.groupby('year_month')['sales_eur'].sum()

# 3. Product Analysis:
top_bu_by_sales = df.groupby('bu')['sales_eur'].sum().nlargest(10)
top_products_by_sales = df.groupby('material')['sales_eur'].sum().nlargest(10)
top_bu_by_profit = df.groupby('bu')['gp_eur'].sum().nlargest(10)
top_products_by_profit = df.groupby('material')['gp_eur'].sum().nlargest(10)

# 4. Customer Analysis:
top_customers_by_sales = df.groupby('customer_name')['sales_eur'].sum().nlargest(10)

# 6. Discount Analysis:
df['discount_eur'] = df['sales_rsp_eur'] - df['sales_eur']
avg_discount = df['discount_eur'].mean()